<a href="https://colab.research.google.com/github/fabian692/DNN-and-preprocesin-cicids-2017/blob/master/New_model_Dense_D_N_N%20TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
pip install pyspark


In [3]:
pip install findspark

In [4]:
pip install sklearn

In [5]:
import numpy as np
import requests
import time
import tensorflow as tf
import keras
import random
import numpy as np
import sklearn
import threading
import requests
import pyspark
import findspark

from keras.optimizers import *
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation

from pyspark import SQLContext, SparkContext
from pyspark import SparkConf

from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.ml.linalg import Vectors



from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold

In [6]:
import numpy as np
from glob import glob
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler

In [7]:
# preprocesamiento de datos
InD = np.zeros((0,79),dtype=object)
for x in glob("/content/drive/My Drive/Colab Notebooks/MachineLearningCVE/*csv"):
    print(x)
    InD=np.vstack((InD,pd.read_csv(x)))

/content/drive/My Drive/Colab Notebooks/MachineLearningCVE/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv
/content/drive/My Drive/Colab Notebooks/MachineLearningCVE/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv
/content/drive/My Drive/Colab Notebooks/MachineLearningCVE/Friday-WorkingHours-Morning.pcap_ISCX.csv
/content/drive/My Drive/Colab Notebooks/MachineLearningCVE/Monday-WorkingHours.pcap_ISCX.csv
/content/drive/My Drive/Colab Notebooks/MachineLearningCVE/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv
/content/drive/My Drive/Colab Notebooks/MachineLearningCVE/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv
/content/drive/My Drive/Colab Notebooks/MachineLearningCVE/Tuesday-WorkingHours.pcap_ISCX.csv
/content/drive/My Drive/Colab Notebooks/MachineLearningCVE/Wednesday-workingHours.pcap_ISCX.csv


In [8]:
Dt=InD[:,:-1].astype(float)

In [9]:
#Remove nan values
LNMV=InD[~np.isnan(Dt).any(axis=1),-1]
DtNMV=Dt[~np.isnan(Dt).any(axis=1)]
#Remove Inf values
LNMIV=LNMV[~np.isinf(DtNMV).any(axis=1)]
DtNMIV=DtNMV[~np.isinf(DtNMV).any(axis=1)]

In [10]:
del(DtNMV)

In [11]:
np.save('NBx', MinMaxScaler().fit_transform(DtNMIV))
np.save('NBy', (LNMIV=='BENIGN').astype(int))
np.save("/content/drive/My Drive/Colab Notebooks/preprocesamiento/",(LNMIV=='BENIGN').astype(int).reshape(-1,1))

In [12]:
DtNMIV.shape

(2827876, 78)

In [13]:
MCDt=DtNMIV[LNMIV!='BENIGN',:]

In [14]:
MCDt.shape

(556556, 78)

In [15]:
MCL=LNMIV[LNMIV!='BENIGN']

In [16]:
MCL.shape

(556556,)

In [17]:
LNMIV.shape,MCL.shape

((2827876,), (556556,))

In [18]:
np.save('NMCx', MinMaxScaler().fit_transform(MCDt))
np.save('NMCy', LabelEncoder().fit_transform(MCL))
np.save("/content/drive/My Drive/Colab Notebooks/preprocesamiento/",OneHotEncoder(sparse=False).fit_transform(MCL.reshape(-1,1)))

In [19]:
#Replace missing values with average and inf values with max
A14=np.average(DtNMIV[:,14])
A15=np.average(DtNMIV[:,15])
M14=np.max(DtNMIV[:,14])
M15=np.max(DtNMIV[:,15])
for c in range(Dt.shape[0]):
    if np.isnan(Dt[c,14]):
        Dt[c,14]=A14
    if np.isnan(Dt[c,15]):
        Dt[c,15]=A15
    if np.isinf(Dt[c,14]):
        Dt[c,14]=M14
    if np.isinf(Dt[c,15]):
        Dt[c,15]=M15

In [20]:
np.save('RBx', MinMaxScaler().fit_transform(Dt))
np.save('RBy', (InD[:,-1]=='BENIGN').astype(int))
np.save("/content/drive/My Drive/Colab Notebooks/preprocesamiento/",(InD[:,-1]=='BENIGN').astype(int).reshape(-1,1))

In [21]:
MCDt=Dt[InD[:,-1]!='BENIGN',:]

In [22]:
MCDt.shape

(557646, 78)

In [23]:
MCL=InD[InD[:,-1]!='BENIGN',-1]

In [24]:
MCL.shape

(557646,)

In [25]:
np.save('RMCx', MinMaxScaler().fit_transform(MCDt))
np.save('RMCy', LabelEncoder().fit_transform(MCL))
np.save("/content/drive/My Drive/Colab Notebooks/preprocesamiento/",OneHotEncoder(sparse=False).fit_transform(MCL.reshape(-1,1)))

In [26]:
# Load the dataset and labels new--
x=np.load('/content/NBx.npy')
y=np.load('/content/NBy.npy')
x.shape,y.shape

((2827876, 78), (2827876,))

In [27]:
conf = SparkConf()
sc = SparkContext(conf = conf)
spark = SQLContext(sc)

In [28]:
# Calculate the rank of each feature
from sklearn.cluster import KMeans
import numpy as np
from sklearn.metrics.cluster import homogeneity_score
# Calculate the rank of each feature
R=[]
for h in range(x.shape[1]):
    kmeans = KMeans(init='k-means++', n_clusters=np.unique(y).shape[0])
    ff=kmeans.fit_predict(x[:,h].reshape(-1,1))
    r=homogeneity_score(y,ff) #Use the homogeneity score as a rank of the feature
    R.append(r)




KeyboardInterrupt: ignored

In [29]:
#Arrange feature accroding to thier ranks
Rnk=np.argsort(np.array(R))

In [30]:
#Initiate the cross-validation splitter
kfolds=StratifiedKFold(n_splits=5,shuffle=True)

In [ ]:
#Per each set of ranks, use cross-validation to calculate accuracy.
smr=[]
for j in range(Rnk.shape[0]):
    fd=x[:,Rnk[j:]]
    pp=0
    lpa=np.zeros((0,2))
    for train,test in kfolds.split(fd,y):
        dff = map(lambda x: (int(float(x[-1])), Vectors.dense(x[:-1])),np.hstack((fd[train],y[train].reshape(-1,1))))
        TrD = spark.createDataFrame(dff,schema=["label", "features"])
        dff = map(lambda x: (int(float(x[-1])), Vectors.dense(x[:-1])),np.hstack((fd[test],y[test].reshape(-1,1))))
        TsD = spark.createDataFrame(dff,schema=["label", "features"])
        model = Sequential()      
        model.add(Dense(1024,input_dim=fd.shape[1],activation='relu'))
        model.add(Dropout(0.01))
        model.add(Dense(768,activation='relu'))
        model.add(Dropout(0.01))
        model.add(Dense(512,activation='relu'))
        model.add(Dropout(0.01))
        model.add(Dense(256,activation='relu'))
        model.add(Dropout(0.01))
        model.add(Dense(128,activation='relu'))
        model.add(Dropout(0.01))
        model.add(Dense(1,activation='sigmoid',use_bias=True)) #The number of neurons is equal to the number of classes
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
        model.summary()
        model.fit(fd[train],y[train] , epochs=1000)
 
       
# evaluamos el modelo
scores = model.evaluate(fd[train],y[train])
 
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
print (model.predict(fd[train]).round())
        

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1024)              11264     
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 768)               787200    
_________________________________________________________________
dropout_1 (Dropout)          (None, 768)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               393728    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               1

In [ ]:
print(R)
print(Rnk)

In [ ]:
smr